In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [2]:
train = pd.read_csv("~/Documents/kaggle/popcorn/data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("~/Documents/kaggle/popcorn/data/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("~/Documents/kaggle/popcorn/data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [3]:
def review_to_wordlist(review, remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub('[^a-zA-Z]', ' ', review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
    return(words)

In [4]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [5]:
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [6]:
sentences = []
for review in train['review']:
    sentences += review_to_sentences(review, tokenizer)
for review in unlabeled_train['review']:
    sentences += review_to_sentences(review, tokenizer)

/home/svet/mainenv/lib/python3.6/site-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/svet/mainenv/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/svet/mainenv/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/svet/mainenv/lib/python3.6/sit

In [7]:
len(sentences)

795538

In [9]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [11]:
from gensim.models import word2vec
model = word2vec.Word2Vec(sentences, workers=num_workers, \
                         size=num_features, min_count=min_word_count, \
                         window=context, sample=downsampling)
model.init_sims(replace=True)
# save the model for later use. You can load it later using Word2Vec.load()
model_name = '300features_4minwords_10context'
model.save(model_name)

2018-12-07 20:54:13,580 : INFO : collecting all words and their counts
2018-12-07 20:54:13,582 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-07 20:54:13,634 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-12-07 20:54:13,684 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-12-07 20:54:13,722 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types
2018-12-07 20:54:13,765 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types
2018-12-07 20:54:13,808 : INFO : PROGRESS: at sentence #50000, processed 1116962 words, keeping 37761 word types
2018-12-07 20:54:13,852 : INFO : PROGRESS: at sentence #60000, processed 1338403 words, keeping 40723 word types
2018-12-07 20:54:13,897 : INFO : PROGRESS: at sentence #70000, processed 1561579 words, keeping 43333 word types
2018-12-07 20:54:13,940 : INFO : PROGRESS: 

2018-12-07 20:54:16,955 : INFO : PROGRESS: at sentence #720000, processed 16105664 words, keeping 118222 word types
2018-12-07 20:54:17,002 : INFO : PROGRESS: at sentence #730000, processed 16332045 words, keeping 118955 word types
2018-12-07 20:54:17,048 : INFO : PROGRESS: at sentence #740000, processed 16553078 words, keeping 119669 word types
2018-12-07 20:54:17,094 : INFO : PROGRESS: at sentence #750000, processed 16771405 words, keeping 120296 word types
2018-12-07 20:54:17,140 : INFO : PROGRESS: at sentence #760000, processed 16990809 words, keeping 120931 word types
2018-12-07 20:54:17,186 : INFO : PROGRESS: at sentence #770000, processed 17217946 words, keeping 121704 word types
2018-12-07 20:54:17,240 : INFO : PROGRESS: at sentence #780000, processed 17448092 words, keeping 122403 word types
2018-12-07 20:54:17,286 : INFO : PROGRESS: at sentence #790000, processed 17675168 words, keeping 123067 word types
2018-12-07 20:54:17,311 : INFO : collected 123505 word types from a corp

2018-12-07 20:55:02,498 : INFO : EPOCH 4 - PROGRESS: at 13.20% examples, 829203 words/s, in_qsize 7, out_qsize 0
2018-12-07 20:55:03,505 : INFO : EPOCH 4 - PROGRESS: at 19.94% examples, 834178 words/s, in_qsize 7, out_qsize 0
2018-12-07 20:55:04,506 : INFO : EPOCH 4 - PROGRESS: at 25.97% examples, 816826 words/s, in_qsize 7, out_qsize 0
2018-12-07 20:55:05,508 : INFO : EPOCH 4 - PROGRESS: at 32.62% examples, 821583 words/s, in_qsize 7, out_qsize 0
2018-12-07 20:55:06,510 : INFO : EPOCH 4 - PROGRESS: at 39.31% examples, 826167 words/s, in_qsize 7, out_qsize 0
2018-12-07 20:55:07,512 : INFO : EPOCH 4 - PROGRESS: at 45.83% examples, 827469 words/s, in_qsize 7, out_qsize 0
2018-12-07 20:55:08,520 : INFO : EPOCH 4 - PROGRESS: at 52.37% examples, 827728 words/s, in_qsize 7, out_qsize 0
2018-12-07 20:55:09,524 : INFO : EPOCH 4 - PROGRESS: at 58.90% examples, 829099 words/s, in_qsize 7, out_qsize 0
2018-12-07 20:55:10,527 : INFO : EPOCH 4 - PROGRESS: at 65.51% examples, 830254 words/s, in_qsiz

In [12]:
model.doesnt_match("man woman child kitchen".split())

/home/svet/mainenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/home/svet/mainenv/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'kitchen'

In [13]:
model.wv.doesnt_match("paris berlin london austria".split())

'paris'

In [14]:
model.most_similar('man')

/home/svet/mainenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6218355894088745),
 ('lad', 0.5990434288978577),
 ('lady', 0.5910966396331787),
 ('monk', 0.5624364614486694),
 ('farmer', 0.5293810963630676),
 ('millionaire', 0.5276523232460022),
 ('guy', 0.5180503726005554),
 ('businessman', 0.5171743631362915),
 ('men', 0.5165083408355713),
 ('chap', 0.5128926038742065)]

In [20]:
model.wv.most_similar('bitchy')

[('feisty', 0.7366248369216919),
 ('ditsy', 0.7355793714523315),
 ('slutty', 0.7325215339660645),
 ('spunky', 0.7284106612205505),
 ('conniving', 0.728180468082428),
 ('perky', 0.7271960973739624),
 ('virginal', 0.7268636226654053),
 ('neurotic', 0.7111197710037231),
 ('bimbo', 0.7092329859733582),
 ('slut', 0.7013981342315674)]

In [21]:
# Load the model that we created in Part 2
# from gensim.models import Word2Vec
# model = Word2Vec.load("300features_40minwords_10context")

In [27]:
import numpy as np
# Function to average all of the word vectors in a given paragraph
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype='float32')
    nwords = 0
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[word])
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [28]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype='float32')
    for review in reviews:
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
    return reviewFeatureVecs

In [29]:
clean_train_reviews = []
for review in train['review']:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

clean_test_reviews = []
for review in test['review']:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

/home/svet/mainenv/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [30]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)

forest = forest.fit( trainDataVecs, train["sentiment"] )
result = forest.predict(testDataVecs)

In [31]:
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv("Word2Vec_AverageVectors.csv", index=False, quoting=3)